# Упругое соударение

- Частицы моделируются кругами радиуса $R$, масса которых равна $1$. 
- У каждой частицы имеется координата $\mathbf{r} = (x, y)$ и скорость $\mathbf{v}=(v_x, v_y)$. 
- За временной шаг $\Delta t$ частица перемещается в соответствии со скоростью $\mathbf{r} ← \mathbf{r} + \mathbf{v}Δ t$.

Пусть $\mathbf r_1=(x_1,y_1)$, $\mathbf r_2=(x_2,y_2)$ – радиус-векторы двух частиц. Тогда расстояние между ними:

$$
d=\sqrt{\Delta x^2+\Delta y^2}
$$

In [2]:
import numpy as np

def get_distance(r1, r2):
    return np.sqrt(np.sum((r1 - r2)**2))

При соударении учитывается, что удар может быть не центральным. Вращение частиц вокруг своей оси не учитывается. Для обновления сокоростей при не центральном ударе можно произвести переход в систему координат (СК), в которой ось $Ox$ проходит через центры частиц в момент контакта. Будем называть эту систему координат собственной СК. В собственной СК упругое столкновение сводится к обмену нормальными компонентами импульсов, а касательные (тангенциальные) компоненты остаются неизменными.

Для перехода в собственную систему отсчета можно использовать матрицу преобразования:

$$
T = 
\begin{bmatrix}
\cos(α)  & \sin(α) \\
-\sin(α) & \cos(α) \\
\end{bmatrix}
$$

Для того, чтобы перейти обратно в глобальную СК, можно использовать обратную матрицу:

$$
T^{(-1)} = 
\begin{bmatrix}
\cos(α)  & -\sin(α) \\
\sin(α) & \cos(α) \\
\end{bmatrix}
$$

$\cos α$ $\sin α$ – комоненты вектора 

$$
\mathbf{n} = \frac{\mathbf{r_1} - \mathbf{r_2}}{∣\mathbf{r_1} - \mathbf{r_2}∣} 
$$

Напишем функцию, возвращающую пару матриц $T$ и $T^{(-1)}$:

In [6]:
def get_transform_pair(r1, r2, distance):
    # (защита от нулевой дистанции)
    if distance == 0:
        n = np.array([1.0, 0.0])
    else:
        n = (r1 - r2) / distance
    cos, sin = n[0], n[1]
    
    transform = np.array([
        [cos,  sin],
        [-sin, cos],
    ])
    reverse = np.array([
        [cos, -sin],
        [sin,  cos],
    ])
    return transform, reverse

Функция, производящая преобразование скоростей при столкновении:

In [7]:
def vels_after_collision(r1, r2, v1, v2, distance):
    transform, reverse = get_transform_pair(r1, r2, distance)

    # Переход в собственную СК
    v1_trans = np.dot(transform, v1)
    v2_trans = np.dot(transform, v2)

    # Обмен скоростями
    v1_trans[0], v2_trans[0] = v2_trans[0], v1_trans[0]
    
    # переход обратно в глобальную СК
    v1_reverse = np.dot(reverse, v1_trans)
    v2_reverse = np.dot(reverse, v2_trans)

    return v1_reverse, v2_reverse

Напишем функцию для одного шага. 

- матрицу всех координат `rs`
- матрицу всех скоростей `vs`
- диаметр частицы `diameter`
- границы среды `bounds` (кортеж)
- ширина стены `wall_width`

Временной шаг $Δt$ примем равным $1$. Тогда обновление координаты:

$$
\mathbf{r} ← \mathbf{r} + \mathbf{v}
$$

Столкновение с внутренней стенкой регистрируется, если частица окажется между границами стенки, т.е. в стенке.

In [36]:
def one_step(rs, vs, diameter, bounds, wall_width):

    # перемещение
    rs += vs

    # столкновение с внешними стенками
    mask = np.logical_or(rs<=bounds[0], rs>=bounds[1])
    rs[mask] -= vs[mask]
    vs[mask] = -vs[mask]
    
    # столкновение с внутренней стенкой
    h_middle = bounds[1] // 2
    half_wall_thickness = wall_width // 2 + diameter // 2
    wall_left = h_middle - half_wall_thickness
    wall_right = h_middle + half_wall_thickness

    mask2 = np.logical_and(rs[:, 0]<=wall_right, rs[:, 0]>=wall_left)
    rs[mask2, 0] -= vs[mask2, 0]
    vs[mask2, 0] = -vs[mask2, 0]
    
    # соударение шариков между собой
    for i in range(len(rs)-1):
        for j in range(i+1, len(rs)):
            r1, r2, v1, v2 = rs[i], rs[j], vs[i], vs[j]
            distance = get_distance(r1, r2)
            if distance <= diameter:
                vs[i], vs[j] = vels_after_collision(r1, r2, v1, v2, distance) 
                rs[i] += vs[i]
                rs[j] += vs[j]

Функция для инициализация стартовых значений координат и скоростей

In [ ]:
def init_rs_vs(bounds, n_particles, velosity_factor=0.01):
    coord_values = np.linspace(bounds[0], bounds[1]//4, int(np.sqrt(n_particles)))
    rs = np.zeros((n_particles, 2), dtype=np.float16)
    counter = 0
    for i in range(int(np.sqrt(n_particles))):
        for j in range(int(np.sqrt(n_particles))):
            rs[counter] = coord_values[i], coord_values[j]
            counter += 1

    vs = np.random.uniform(-bounds[1], bounds[1], size=(n_particles, 2)) * velosity_factor
    return rs, vs

# Запуск симуляции

In [60]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle, Circle

%matplotlib qt

bounds = 0., 100.   # границы (совпадают как для X так и для Y)
wall_width = 2
N = 50              # число частиц
d = 2.

rs, vs = init_rs_vs(bounds, N, velosity_factor=0.02)

GRAY = "#333"
DARKGRAY = "#222"
figwidth = 10

fig, ax = plt.subplots()
fig.set_figwidth(figwidth)
fig.set_figheight(figwidth)
fig.set_facecolor(DARKGRAY)

# Палитра/нормализация для стабильной окраски по скорости
cmap = plt.get_cmap('hot')
speed_norm = mcolors.Normalize(vmin=0, vmax=np.linalg.norm(vs, axis=1).max() + 1e-12)

for n in range(500):
    ax.clear()

    # Cерая полоса-стена по центру (стена)
    h_middle = (bounds[0] + bounds[1]) / 2
    wall_x0 = h_middle - wall_width / 2
    ax.add_patch(
        Rectangle(
            (wall_x0, bounds[0]),
            width=wall_width,
            height=bounds[1] - bounds[0],
            facecolor=DARKGRAY, edgecolor='none', zorder=0
        )
    )

    # Частицы как круги радиусом d/2
    colors = np.linalg.norm(vs, axis=1) # Скорости для окраски
    for (x, y), c in zip(rs, colors):
        ax.add_patch(
            Circle(
                (x, y),
                radius=d/2,
                facecolor=cmap(speed_norm(c)),
                edgecolor='none',
                zorder=1
            )
        )

    one_step(rs, vs, d, bounds, wall_width)

    ax.set_xlim(bounds)
    ax.set_ylim(bounds)
    ax.set_aspect('equal', adjustable='box')
    ax.set_facecolor(GRAY)

    # Убираем подписи и риски на осях
    ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.pause(0.05)

# Запись анимации в MP4

In [70]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle, Circle
from matplotlib.animation import FFMpegWriter
import numpy as np
import matplotlib as mpl
import imageio_ffmpeg


bounds = 0., 100.
wall_width = 2
N = 50
d = 2.
rs, vs = init_rs_vs(bounds, N, velosity_factor=0.021)

GRAY = "#333"
DARKGRAY = "#222"
figwidth = 10

plt.ioff()  # выключаем интерактив, чтобы ничего не мигало во время записи
fig, ax = plt.subplots()
fig.set_figwidth(figwidth)
fig.set_figheight(figwidth)
fig.set_facecolor(DARKGRAY)

cmap = plt.get_cmap('hot')
speed_norm = mcolors.Normalize(vmin=0, vmax=np.linalg.norm(vs, axis=1).max() + 1e-12)

# Настройка видеозаписи
fps = 15
frames = 1200
writer = FFMpegWriter(
    fps=fps,
    codec='libx264',          # H.264
    bitrate=1000,             # можно поднять до 6000–8000 для лучшего качества
    metadata={'title': '2D elastic collisions', 'artist': 'ordevoir'}
)

out_path = "./simulation.mp4"
dpi = 150
mpl.rcParams['animation.ffmpeg_path'] = imageio_ffmpeg.get_ffmpeg_exe()

with writer.saving(fig, out_path, dpi):
    for n in range(frames):
        ax.clear()

        # центральная «стенка» как полоса
        h_middle = (bounds[0] + bounds[1]) / 2
        wall_x0 = h_middle - wall_width / 2
        ax.add_patch(
            Rectangle(
                (wall_x0, bounds[0]),
                width=wall_width,
                height=bounds[1] - bounds[0],
                facecolor=DARKGRAY, edgecolor='none', zorder=0
            )
        )

        # частицы как круги радиусом d/2
        colors = np.linalg.norm(vs, axis=1)
        for (x, y), c in zip(rs, colors):
            ax.add_patch(
                Circle(
                    (x, y),
                    radius=d/2,
                    facecolor=cmap(speed_norm(c)),
                    edgecolor='none',
                    zorder=1
                )
            )

        one_step(rs, vs, d, bounds, wall_width)

        ax.set_xlim(bounds)
        ax.set_ylim(bounds)
        ax.set_aspect('equal', adjustable='box')
        ax.set_facecolor(GRAY)
        ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)
        for spine in ax.spines.values():
            spine.set_visible(False)

        
        writer.grab_frame()     # записываем кадр

plt.close(fig)
print(f"Path: {out_path}")

Path: ./simulation.mp4


In [ ]:
import subprocess
from pathlib import Path
from typing import Union
import imageio_ffmpeg

Time = Union[float, int, str]  # секунды (число) или "HH:MM:SS[.ms]"

def _to_ffmpeg_ts(t: Time) -> str:
    """Привести время к строке HH:MM:SS.mmm для ffmpeg."""
    if isinstance(t, (int, float)):
        ms = int(round((t - int(t)) * 1000))
        s  = int(t) % 60
        m  = (int(t) // 60) % 60
        h  = int(t) // 3600
        return f"{h:02d}:{m:02d}:{s:02d}.{ms:03d}"
    return t  # уже строка

def trim_mp4(
    in_path: str,
    out_path: str,
    start: Time,
    end: Time,
    accurate: bool = False,
    reencode_video: str = "libx264",  # используется если accurate=True
    reencode_audio: str = "aac",      # используется если accurate=True
    crf: int = 18,                    # качество для H.264 (меньше = лучше)
    preset: str = "medium"            # скорость/сжатие для H.264
) -> None:
    """
    Обрезать видео [start, end). Если accurate=False — быстрое копирование потоков,
    но резка по ближайшим ключевым кадрам. Если accurate=True — точная резка с перекодированием.
    """
    ffmpeg = imageio_ffmpeg.get_ffmpeg_exe()
    in_path = str(Path(in_path))
    out_path = str(Path(out_path))
    ss = _to_ffmpeg_ts(start)
    to = _to_ffmpeg_ts(end)

    # Для ffmpeg удобнее задавать длительность (-t), чем -to, когда используем -ss.
    # Посчитаем длительность в секундах для надёжности.
    def _to_seconds(t: Time) -> float:
        if isinstance(t, (int, float)):
            return float(t)
        h, m, s = t.split(":")
        s = float(s)
        return int(h)*3600 + int(m)*60 + s

    duration = _to_seconds(end) - _to_seconds(start)
    if duration <= 0:
        raise ValueError("end должно быть больше start")

    if accurate:
        # Точный вариант: -ss после -i и перекодирование
        cmd = [
            ffmpeg, "-y",
            "-i", in_path,
            "-ss", ss,
            "-t", f"{duration:.3f}",
            "-map", "0",
            "-c:v", reencode_video, "-crf", str(crf), "-preset", preset,
            "-c:a", reencode_audio, "-b:a", "192k",
            "-movflags", "+faststart",
            out_path
        ]
    else:
        # Быстрый вариант: -ss до -i и копирование потоков (режет по keyframe)
        cmd = [
            ffmpeg, "-y",
            "-ss", ss,
            "-i", in_path,
            "-t", f"{duration:.3f}",
            "-map", "0",
            "-c", "copy",
            "-avoid_negative_ts", "make_zero",
            "-movflags", "+faststart",
            out_path
        ]

    print("Running:", " ".join(cmd))
    completed = subprocess.run(cmd, capture_output=True, text=True)
    if completed.returncode != 0:
        # Вывести stderr для диагностики
        raise RuntimeError(f"ffmpeg error:\n{completed.stderr}")
    else:
        print(f"Saved: {out_path}")

# Примеры использования:

# 1) Быстро (без перекодирования), с 10.0 c до 25.0 c:
# trim_mp4("./simulation.mp4", "./simulation_trim_fast.mp4", start=10.0, end=25.0, accurate=False)

# 2) Точно (с перекодированием), с 00:00:10.000 до 00:00:25.000:
# trim_mp4("./simulation.mp4", "./simulation_trim_exact.mp4",
#          start="00:00:10.000", end="00:00:25.000", accurate=True, crf=20, preset="faster")


In [72]:
trim_mp4(out_path, "simul.mp4", 0, 15)

Running: c:\Users\werna\miniconda3\envs\marl\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -y -ss 00:00:00.000 -i simulation.mp4 -t 15.000 -map 0 -c copy -avoid_negative_ts make_zero -movflags +faststart simul.mp4
Saved: simul.mp4
